# Import Libraries

In [1]:
import numpy as np
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings("ignore")

# Loading Data

In [3]:
import os
cwd = os.getcwd()
data_path = os.path.join(cwd, "data")

In [4]:
loan_data_inputs_train = pd.read_csv(data_path + "/" + "loan_data_X_train.csv", index_col = 0)
loan_data_targets_train = pd.read_csv(data_path + "/" + "loan_data_y_train.csv", index_col = 0, header = None)
loan_data_inputs_test = pd.read_csv(data_path + "/" + "loan_data_X_test.csv", index_col = 0)
loan_data_targets_test = pd.read_csv(data_path + "/" + "loan_data_y_test.csv", index_col = 0, header = None)

In [5]:
loan_data_backup = pd.read_csv(data_path + "/" + "loan_data_2007_2014.csv")

In [6]:
loan_data = loan_data_backup.copy()

# Explore Data

In [7]:
pd.options.display.max_columns = None

In [8]:
loan_data.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-85,1.0,NaN,NaN,3.0,0.0,13648,83.7,9.0,f,0.0,0.0,5861.071414,5831.78,5000.00,861.07,0.00,0.00,0.00,Jan-15,171.62,NaN,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-99,5.0,NaN,NaN,3.0,0.0,1687,9.4,4.0,f,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,NaN,Sep-13,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,small_business,real estate business,606xx,IL,8.72,0.0,Nov-01,2.0,NaN,NaN,2.0,0.0,2956,98.5,10.0,f,0.0,0.0,3003.653644,3003.65,2400.00,603.65,0.00,0.00,0.00,Jun-14,649.91,NaN,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0.0,Feb-96,1.0,35.0,NaN,10.0,0.0,5598,21.0,37.0,f,0.0,0.0,12226.302210,12226.30,10000.00,2209.33,16.97,0.00,0.00,Jan-15,357.48,NaN,Jan-15,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-11,Current,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > I plan on combi...,other,Personal,972xx,OR,17.94,0.0,Jan-96,0.0,38.0,NaN,15.0,0.0,27783,53.9,38.0,f,766.9,766.9,3242.170000,3242.17,2233.10,1009.07,0.00,0.00,0.00,Jan-16,67.79,Feb-16,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
loan_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466285 entries, 0 to 466284
Data columns (total 75 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Unnamed: 0                   466285 non-null  int64  
 1   id                           466285 non-null  int64  
 2   member_id                    466285 non-null  int64  
 3   loan_amnt                    466285 non-null  int64  
 4   funded_amnt                  466285 non-null  int64  
 5   funded_amnt_inv              466285 non-null  float64
 6   term                         466285 non-null  object 
 7   int_rate                     466285 non-null  float64
 8   installment                  466285 non-null  float64
 9   grade                        466285 non-null  object 
 10  sub_grade                    466285 non-null  object 
 11  emp_title                    438697 non-null  object 
 12  emp_length                   445277 non-null  object 
 13 

# Population Stability Index: Preprocessing

Copy codes from the Data Preparation notebook with minor adjustments:
data preprocessing, fine classing, and coarse classing on the new data, in order to calculate statistics for the exact same variables to the ones we used for training and testing the PD model.

# General Preprocessing

## Preprocessing few continuous variables

### Convert string to numeric

In [10]:
loan_data['emp_length'].unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', nan],
      dtype=object)

In [11]:
loan_data['emp_length_int'] = loan_data['emp_length'].str.replace('\+ years','')
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace('< 1 year',str(0))
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace('n/a',str(0))
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace(' years','')
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace(' year','')

In [12]:
type(loan_data['emp_length_int'][0])

str

In [13]:
loan_data['emp_length_int'] = pd.to_numeric(loan_data['emp_length_int'])

In [14]:
type(loan_data['emp_length_int'][0])

numpy.float64

In [15]:
loan_data['term'].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [16]:
loan_data['term_int'] = loan_data['term'].str.replace(' months','')

In [17]:
loan_data['term_int'] = pd.to_numeric(loan_data['term_int'])

In [18]:
type(loan_data['term_int'][0])

numpy.int64

### Convert to datetime

In [19]:
loan_data['earliest_cr_line']

0         Jan-85
1         Apr-99
2         Nov-01
3         Feb-96
4         Jan-96
           ...  
466280    Apr-03
466281    Jun-97
466282    Dec-01
466283    Feb-03
466284    Feb-00
Name: earliest_cr_line, Length: 466285, dtype: object

In [20]:
loan_data['earliest_cr_line_date'] = pd.to_datetime(loan_data['earliest_cr_line'],
                                                   format = '%b-%y')

In [21]:
type(loan_data['earliest_cr_line_date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [22]:
loan_data['issue_date'] = pd.to_datetime(loan_data['issue_d'],
                                                   format = '%b-%y')

In [23]:
loan_data['issue_date'].head(5)

0   2011-12-01
1   2011-12-01
2   2011-12-01
3   2011-12-01
4   2011-12-01
Name: issue_date, dtype: datetime64[ns]

### Add new feature

In [24]:
pd.to_datetime('2019-01-01') - loan_data['earliest_cr_line_date']

0        12418 days
1         7215 days
2         6270 days
3         8370 days
4         8401 days
            ...    
466280    5754 days
466281    7884 days
466282    6240 days
466283    5813 days
466284    6909 days
Name: earliest_cr_line_date, Length: 466285, dtype: timedelta64[ns]

In [25]:
loan_data['mths_since_earliest_cr_line'] = round(
    pd.to_numeric(
        (pd.to_datetime('2019-01-01') - loan_data['earliest_cr_line_date']) 
        / np.timedelta64(1,'M') # precise conversion to month
))

In [26]:
loan_data['mths_since_issue_d'] = round(
    pd.to_numeric(
        (pd.to_datetime('2019-01-01') - loan_data['issue_date']) 
        / np.timedelta64(1,'M') # precise conversion to month
))

### Clean wrong data < 0

In [27]:
loan_data['mths_since_earliest_cr_line'].describe()

count    466256.000000
mean        252.482430
std          93.974829
min        -599.000000
25%         196.000000
50%         238.000000
75%         298.000000
max         600.000000
Name: mths_since_earliest_cr_line, dtype: float64

In [28]:
loan_data.loc[: ,
              ['earliest_cr_line','earliest_cr_line_date',
               'mths_since_earliest_cr_line']]\
    [loan_data['mths_since_earliest_cr_line'] < 0]

,earliest_cr_line,earliest_cr_line_date,mths_since_earliest_cr_line
1580,Sep-62,2062-09-01,-524.0
1770,Sep-68,2068-09-01,-596.0
2799,Sep-64,2064-09-01,-548.0
3282,Sep-67,2067-09-01,-584.0
3359,Feb-65,2065-02-01,-553.0
...,...,...,...
464003,Jan-68,2068-01-01,-588.0
464260,Jul-66,2066-07-01,-570.0
465100,Oct-67,2067-10-01,-585.0
465500,Sep-67,2067-09-01,-584.0


In [29]:
loan_data['mths_since_earliest_cr_line'][loan_data['mths_since_earliest_cr_line']<0]=\
loan_data['mths_since_earliest_cr_line'].max()

In [30]:
min(loan_data['mths_since_earliest_cr_line'])

86.0

In [31]:
loan_data['earliest_cr_line_date'].describe()

count                  466256
unique                    664
top       2000-10-01 00:00:00
freq                     3674
first     1969-01-01 00:00:00
last      2068-12-01 00:00:00
Name: earliest_cr_line_date, dtype: object

In [32]:
loan_data['mths_since_issue_d'].describe()

count    466285.000000
mean         64.255187
std          14.340154
min          49.000000
25%          54.000000
50%          60.000000
75%          70.000000
max         139.000000
Name: mths_since_issue_d, dtype: float64

In [33]:
loan_data['issue_date'].describe()

count                  466285
unique                     91
top       2014-10-01 00:00:00
freq                    38782
first     2007-06-01 00:00:00
last      2014-12-01 00:00:00
Name: issue_date, dtype: object

## Preprocessing few discrete variables: create dummies

In [34]:
loan_data_dummies = [
    pd.get_dummies(loan_data['grade'],prefix='grade',prefix_sep=':'),
    pd.get_dummies(loan_data['sub_grade'],prefix='sub_grade',prefix_sep=':'),    
    pd.get_dummies(loan_data['home_ownership'],prefix='home_ownership',prefix_sep=':'),    
    pd.get_dummies(loan_data['verification_status'],prefix='verification_status',prefix_sep=':'),    
    pd.get_dummies(loan_data['loan_status'],prefix='loan_status',prefix_sep=':'),    
    pd.get_dummies(loan_data['purpose'],prefix='purpose',prefix_sep=':'),    
    pd.get_dummies(loan_data['addr_state'],prefix='addr_state',prefix_sep=':'),    
    pd.get_dummies(loan_data['initial_list_status'],prefix='initial_list_status',prefix_sep=':') 
]

In [35]:
loan_data_dummies = pd.concat(loan_data_dummies, axis = 1)

In [36]:
type(loan_data_dummies)

pandas.core.frame.DataFrame

In [37]:
loan_data = pd.concat([loan_data, loan_data_dummies], axis=1) 
# axix = 0 by row, axis = 1 by col

In [38]:
loan_data.columns.values

array(['Unnamed: 0', 'id', 'member_id', 'loan_amnt', 'funded_amnt',
       'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade',
       'sub_grade', 'emp_title', 'emp_length', 'home_ownership',
       'annual_inc', 'verification_status', 'issue_d', 'loan_status',
       'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
       'recoveries', 'collection_recovery_fee', 'last_pymnt_d',
       'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
       'veri

## Check for missing values and clean

In [39]:
pd.options.display.max_rows = None
loan_data.isnull().sum()

Unnamed: 0                                                              0
id                                                                      0
member_id                                                               0
loan_amnt                                                               0
funded_amnt                                                             0
funded_amnt_inv                                                         0
term                                                                    0
int_rate                                                                0
installment                                                             0
grade                                                                   0
sub_grade                                                               0
emp_title                                                           27588
emp_length                                                          21008
home_ownership                        

In [40]:
pd.options.display.max_rows = 100

### total revolving limit

In [41]:
# revolving limit: if missing, assume it is equal to funded amount of the loan
loan_data['total_rev_hi_lim'] = loan_data['total_rev_hi_lim'].fillna(loan_data['funded_amnt'])

In [42]:
loan_data['total_rev_hi_lim'].isnull().sum()

0

In [43]:
loan_data.loc[:,['total_rev_hi_lim','funded_amnt']][loan_data['total_rev_hi_lim'].isnull()]

,total_rev_hi_lim,funded_amnt


### others

In [44]:
loan_data['annual_inc'].fillna(loan_data['annual_inc'].mean(), inplace=True)
loan_data['mths_since_earliest_cr_line'].fillna(0, inplace=True)
loan_data['acc_now_delinq'].fillna(0, inplace=True)
loan_data['total_acc'].fillna(0, inplace=True)
loan_data['pub_rec'].fillna(0, inplace=True)
loan_data['open_acc'].fillna(0, inplace=True)
loan_data['inq_last_6mths'].fillna(0, inplace=True)
loan_data['delinq_2yrs'].fillna(0, inplace=True)
loan_data['emp_length_int'].fillna(0, inplace=True)

# Dependent Variable 

## Default Definition: 1 for good, 0 for bad

In [45]:
loan_data['loan_status'].unique()

array(['Fully Paid', 'Charged Off', 'Current', 'Default',
       'Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off'],
      dtype=object)

In [46]:
loan_data['loan_status'].value_counts() / loan_data['loan_status'].count()

Current                                                0.480878
Fully Paid                                             0.396193
Charged Off                                            0.091092
Late (31-120 days)                                     0.014798
In Grace Period                                        0.006747
Does not meet the credit policy. Status:Fully Paid     0.004263
Late (16-30 days)                                      0.002612
Default                                                0.001784
Does not meet the credit policy. Status:Charged Off    0.001632
Name: loan_status, dtype: float64

In [47]:
# use np.where as if-statement
loan_data['good_bad'] = np.where(loan_data['loan_status'].isin([
        'Charged Off',
        'Default',
        'Does not meet the credit policy. Status:Charged Off',
        'Late (31-120 days)']), 0, 1)

In [48]:
loan_data['good_bad'].value_counts()

1    415317
0     50968
Name: good_bad, dtype: int64

# Save data

In [49]:
loan_data.to_csv(data_path + "/" + "loan_data_new.csv", index=False)

# Modeling

## (Deleted) Split Data

In [50]:
# X_train_data,X_test_data,y_train_data,y_test_data = train_test_split(loan_data.drop('good_bad',axis=1), 
#                                                  loan_data['good_bad'],
#                                                  test_size=0.2,
#                                                  random_state=42)

In [51]:
# X_train.shape

In [52]:
# X_test.shape

In [53]:
# y_train.shape

In [54]:
# y_test.shape

## Preprocessing Discrete Variables

In [55]:
X_train = loan_data.drop('good_bad', axis=1)
y_train = loan_data['good_bad']

### An Example: WoE and IV

In [56]:
df1 = pd.concat([X_train['grade'], y_train], axis=1)
df1.head()

,grade,good_bad
0,B,1
1,C,0
2,C,1
3,C,1
4,B,1


In [57]:
df1.groupby(df1.columns.values[0], as_index = False)[df1.columns.values[1]].count()
# df1.groupby(['grade'], as_index = False).count()

,grade,good_bad
0,A,74867
1,B,136929
2,C,125293
3,D,76888
4,E,35757
5,F,13229
6,G,3322


In [58]:
df1 = pd.concat([df1.groupby(['grade'], as_index = False)['good_bad'].count(),
                 df1.groupby(['grade'], as_index = False)['good_bad'].mean()],
                 axis = 1)
df1 = df1.iloc[:, [0, 1, 3]]

In [59]:
df1.columns = ['grade', 'n_obs', 'prop_good']

In [60]:
df1['prop_n_obs'] = df1['n_obs'] / df1['n_obs'].sum()

In [61]:
df1

,grade,n_obs,prop_good,prop_n_obs
0,A,74867,0.961305,0.160561
1,B,136929,0.921426,0.293659
2,C,125293,0.885197,0.268705
3,D,76888,0.845906,0.164895
4,E,35757,0.805241,0.076685
5,F,13229,0.758712,0.028371
6,G,3322,0.721854,0.007124


In [62]:
df1['n_good'] = df1['prop_good'] * df1['n_obs']
df1['n_bad'] = (1 - df1['prop_good']) * df1['n_obs']

In [63]:
df1['prop_n_good'] = df1['n_good'] / df1['n_good'].sum()
df1['prop_n_bad'] = df1['n_bad'] / df1['n_bad'].sum()

In [64]:
df1['WoE'] = np.log(df1['prop_n_good'] / df1['prop_n_bad'])

In [65]:
df1 = df1.sort_values(['WoE'])
df1 = df1.reset_index(drop = True)
df1

,grade,n_obs,prop_good,prop_n_obs,n_good,n_bad,prop_n_good,prop_n_bad,WoE
0,G,3322,0.721854,0.007124,2398.0,924.0,0.005774,0.018129,-1.144166
1,F,13229,0.758712,0.028371,10037.0,3192.0,0.024167,0.062628,-0.952214
2,E,35757,0.805241,0.076685,28793.0,6964.0,0.069328,0.136635,-0.678466
3,D,76888,0.845906,0.164895,65040.0,11848.0,0.156603,0.232460,-0.395001
4,C,125293,0.885197,0.268705,110909.0,14384.0,0.267047,0.282216,-0.055251
5,B,136929,0.921426,0.293659,126170.0,10759.0,0.303792,0.211093,0.364043
6,A,74867,0.961305,0.160561,71970.0,2897.0,0.173289,0.056840,1.114730


In [66]:
df1['dff_prop_good'] = df1['prop_good'].diff().abs()
df1['diff_WoE'] = df1['WoE'].diff().abs()
df1

,grade,n_obs,prop_good,prop_n_obs,n_good,n_bad,prop_n_good,prop_n_bad,WoE,dff_prop_good,diff_WoE
0,G,3322,0.721854,0.007124,2398.0,924.0,0.005774,0.018129,-1.144166,NaN,NaN
1,F,13229,0.758712,0.028371,10037.0,3192.0,0.024167,0.062628,-0.952214,0.036858,0.191952
2,E,35757,0.805241,0.076685,28793.0,6964.0,0.069328,0.136635,-0.678466,0.046529,0.273748
3,D,76888,0.845906,0.164895,65040.0,11848.0,0.156603,0.232460,-0.395001,0.040665,0.283465
4,C,125293,0.885197,0.268705,110909.0,14384.0,0.267047,0.282216,-0.055251,0.039291,0.339750
5,B,136929,0.921426,0.293659,126170.0,10759.0,0.303792,0.211093,0.364043,0.036229,0.419294
6,A,74867,0.961305,0.160561,71970.0,2897.0,0.173289,0.056840,1.114730,0.039878,0.750686


In [67]:
df1['IV'] = (df1['prop_n_good'] - df1['prop_n_bad']) * df1['WoE']
df1['IV'] = df1['IV'].sum()
df1

,grade,n_obs,prop_good,prop_n_obs,n_good,n_bad,prop_n_good,prop_n_bad,WoE,dff_prop_good,diff_WoE,IV
0,G,3322,0.721854,0.007124,2398.0,924.0,0.005774,0.018129,-1.144166,NaN,NaN,0.290782
1,F,13229,0.758712,0.028371,10037.0,3192.0,0.024167,0.062628,-0.952214,0.036858,0.191952,0.290782
2,E,35757,0.805241,0.076685,28793.0,6964.0,0.069328,0.136635,-0.678466,0.046529,0.273748,0.290782
3,D,76888,0.845906,0.164895,65040.0,11848.0,0.156603,0.232460,-0.395001,0.040665,0.283465,0.290782
4,C,125293,0.885197,0.268705,110909.0,14384.0,0.267047,0.282216,-0.055251,0.039291,0.339750,0.290782
5,B,136929,0.921426,0.293659,126170.0,10759.0,0.303792,0.211093,0.364043,0.036229,0.419294,0.290782
6,A,74867,0.961305,0.160561,71970.0,2897.0,0.173289,0.056840,1.114730,0.039878,0.750686,0.290782


### Automating Calculations: WoE & IV

In [68]:
def woe_discrete(df, x, y):
    df = pd.concat([df[x], y], axis = 1)
    df = pd.concat([
        df.groupby(df.columns.values[0], as_index = False)[df.columns.values[1]].count(),
        df.groupby(df.columns.values[0], as_index = False)[df.columns.values[1]].mean()
        ], axis = 1)
    df = df.iloc[:, [0, 1, 3]]
    df.columns = [df.columns.values[0], 'n_obs', 'prop_good']
    df['prop_n_obs'] = df['n_obs'] / df['n_obs'].sum()
    df['n_good'] = df['prop_good'] * df['n_obs']
    df['n_bad'] = (1 - df['prop_good']) * df['n_obs']
    df['prop_n_good'] = df['n_good'] / df['n_good'].sum()
    df['prop_n_bad'] = df['n_bad'] / df['n_bad'].sum()
    df['WoE'] = np.log(df['prop_n_good'] / df['prop_n_bad'])
    df = df.sort_values(['WoE'])
    df = df.reset_index(drop = True)
    df['dff_prop_good'] = df['prop_good'].diff().abs()
    df['diff_WoE'] = df['WoE'].diff().abs()
    df['IV'] = (df['prop_n_good'] - df['prop_n_bad']) * df['WoE']
    df['IV'] = df['IV'].sum()
    return df

In [69]:
# df_temp = woe_discrete(X_test, 'grade', y_test)
# df_temp

### Visualizing Results

In [70]:
# overwrite the default maplotlib look with seaborn
# sns.set()

In [71]:
def plot_by_woe(df_woe, rotation_of_x_axis_labels = 0):
    x = np.array(df_woe.iloc[:, 0].apply(str)) # turn into text strings in ndarray
    y = df_woe['WoE']
    # formatting
    plt.figure(figsize = (16, 6))
    plt.plot(x, y, marker = 'o', linestyle = '--', color = 'k')
    plt.xlabel(df_woe.columns[0])
    plt.ylabel('Weight of Evidence')
    plt.title(str('WoE by ' + df_woe.columns[0]))
    plt.xticks(rotation = rotation_of_x_axis_labels)

In [72]:
# plot_by_woe(df_temp)

### Creating Dummy Variables

#### home ownership

In [73]:
# df_temp = woe_discrete(X_train, 'home_ownership', y_train)
# df_temp

In [74]:
# plot_by_woe(df_temp)

In [75]:
X_train['home_ownership:RENT_OTHER_NONE_ANY'] = sum([X_train['home_ownership:RENT'],
                                                    X_train['home_ownership:OTHER'],
                                                    X_train['home_ownership:NONE'],
                                                    X_train['home_ownership:ANY']])

#### address state

In [76]:
# df_temp = woe_discrete(X_train, 'addr_state', y_train)
# df_temp

In [77]:
# plot_by_woe(df_temp)

state ND is missing in our data. add it as a dummy variable in case it is used.

In [78]:
if ['addr_state:ND'] in X_train.columns.values:
    pass
else:
    X_train['addr_state:ND'] = 0

In [79]:
#plot_by_woe(df_temp.iloc[1: -2, :])

In [80]:
# plot_by_woe(df_temp.iloc[6: -6, :])

In [81]:
X_train['addr_state:ND_IA_AL_AR_MS_OK_LA'] = sum([
                           X_train['addr_state:ND'],
                           X_train['addr_state:IA'],
                           X_train['addr_state:AL'],
                           X_train['addr_state:AR'],
                           X_train['addr_state:MS'],
                           X_train['addr_state:OK'],
                           X_train['addr_state:LA']])
X_train['addr_state:NV_NM_HI_SD'] = sum([
                           X_train['addr_state:NV'],
                           X_train['addr_state:NM'],
                           X_train['addr_state:HI'],
                           X_train['addr_state:SD']])
X_train['addr_state:MO_MD_NC'] = sum([
                           X_train['addr_state:MO'],
                           X_train['addr_state:MD'],
                           X_train['addr_state:NC']])
X_train['addr_state:PA_KY_TN'] = sum([
                           X_train['addr_state:PA'],
                           X_train['addr_state:KY'],
                           X_train['addr_state:TN']])
X_train['addr_state:NJ_MI_IN'] = sum([
                           X_train['addr_state:NJ'],
                           X_train['addr_state:MI'],
                           X_train['addr_state:IN']])
X_train['addr_state:VA_MN_AK'] = sum([
                           X_train['addr_state:VA'],
                           X_train['addr_state:MN'],
                           X_train['addr_state:AK']])
X_train['addr_state:AZ_NE_OH'] = sum([
                           X_train['addr_state:AZ'],
                           X_train['addr_state:NE'],
                           X_train['addr_state:OH']])
X_train['addr_state:DE_MA_UT_WI'] = sum([
                           X_train['addr_state:DE'],
                           X_train['addr_state:MA'],
                           X_train['addr_state:UT'],
                           X_train['addr_state:WI']])
X_train['addr_state:GA_RI_WY_IL_MT_KS'] = sum([
                           X_train['addr_state:GA'],
                           X_train['addr_state:RI'],
                           X_train['addr_state:WY'],
                           X_train['addr_state:IL'],
                           X_train['addr_state:MT'],
                           X_train['addr_state:KS']])
X_train['addr_state:CT_ND_WA_CO_SC'] = sum([
                           X_train['addr_state:CT'],
                           X_train['addr_state:ND'],
                           X_train['addr_state:WA'],
                           X_train['addr_state:CO'],
                           X_train['addr_state:SC']])
X_train['addr_state:WV_OR_NH_DC_ID_VT_ME'] = sum([
                           X_train['addr_state:WV'],
                           X_train['addr_state:OR'],
                           X_train['addr_state:NH'],
                           X_train['addr_state:DC'],
                           X_train['addr_state:ID'],
                           X_train['addr_state:VT'],
                           X_train['addr_state:ME']])

## Preprocessing Continuous Variables

### Automating Calculations and Visualizing Results

In [82]:
def woe_ordered_continuous(df, x, y):
    df = pd.concat([df[x], y], axis = 1)
    df = pd.concat([
        df.groupby(df.columns.values[0], as_index = False)[df.columns.values[1]].count(),
        df.groupby(df.columns.values[0], as_index = False)[df.columns.values[1]].mean()
        ], axis = 1)
    df = df.iloc[:, [0, 1, 3]]
    df.columns = [df.columns.values[0], 'n_obs', 'prop_good']
    df['prop_n_obs'] = df['n_obs'] / df['n_obs'].sum()
    df['n_good'] = df['prop_good'] * df['n_obs']
    df['n_bad'] = (1 - df['prop_good']) * df['n_obs']
    df['prop_n_good'] = df['n_good'] / df['n_good'].sum()
    df['prop_n_bad'] = df['n_bad'] / df['n_bad'].sum()
    df['WoE'] = np.log(df['prop_n_good'] / df['prop_n_bad'])
    #df = df.sort_values(['WoE'])
    #df = df.reset_index(drop = True)
    df['dff_prop_good'] = df['prop_good'].diff().abs()
    df['diff_WoE'] = df['WoE'].diff().abs()
    df['IV'] = (df['prop_n_good'] - df['prop_n_bad']) * df['WoE']
    df['IV'] = df['IV'].sum()
    return df

### Creating Dummy Variables

#### term

In [83]:
# df_temp = woe_ordered_continuous(X_train, 'term_int', y_train)
# df_temp

In [84]:
# plot_by_woe(df_temp)

In [85]:
X_train['term:36'] = np.where((X_train['term_int'] == 36), 1, 0)
X_train['term:60'] = np.where((X_train['term_int'] == 60), 1, 0)

#### employment length

In [86]:
# df_temp = woe_ordered_continuous(X_train, 'emp_length_int', y_train)
# df_temp

In [87]:
# plot_by_woe(df_temp)

In [88]:
X_train['emp_length:0'] = np.where((X_train['emp_length_int'].isin([0])), 1, 0)
X_train['emp_length:1'] = np.where((X_train['emp_length_int'].isin([1])), 1, 0)
X_train['emp_length:2-6'] = np.where((X_train['emp_length_int'].isin(range(2, 7))), 1, 0)
X_train['emp_length:7-9'] = np.where((X_train['emp_length_int'].isin(range(7, 10))), 1, 0)
X_train['emp_length:10'] = np.where((X_train['emp_length_int'].isin([10])), 1, 0)

#### months since issue date

In [89]:
X_train['mths_since_issue_d_factor'] = pd.cut(X_train['mths_since_issue_d'], 50 )

In [90]:
# df_temp = woe_ordered_continuous(X_train, 'mths_since_issue_d_factor', y_train)
# df_temp

In [91]:
# plot_by_woe(df_temp, 90)

In [92]:
#plot_by_woe(df_temp.iloc[15: , :], 90)

In [93]:
X_train['mths_since_issue_d:<4'] = np.where((X_train['mths_since_issue_d'].isin(range(4))), 1, 0)
X_train['mths_since_issue_d:4-6'] = np.where((X_train['mths_since_issue_d'].isin(range(4,7))), 1, 0)
X_train['mths_since_issue_d:7-9'] = np.where((X_train['mths_since_issue_d'].isin(range(7,10))), 1, 0)
X_train['mths_since_issue_d:10-12'] = np.where((X_train['mths_since_issue_d'].isin(range(10,13))), 1, 0)
X_train['mths_since_issue_d:13-14'] = np.where((X_train['mths_since_issue_d'].isin(range(13,15))), 1, 0)
X_train['mths_since_issue_d:15-17'] = np.where((X_train['mths_since_issue_d'].isin(range(15,18))), 1, 0)
X_train['mths_since_issue_d:18-23'] = np.where((X_train['mths_since_issue_d'].isin(range(18,24))), 1, 0)
X_train['mths_since_issue_d:24-28'] = np.where((X_train['mths_since_issue_d'].isin(range(24,29))), 1, 0)
X_train['mths_since_issue_d:29-58'] = np.where((X_train['mths_since_issue_d'].isin(range(29,59))), 1, 0)
X_train['mths_since_issue_d:59-75'] = np.where((X_train['mths_since_issue_d'].isin(range(59,76))), 1, 0)
X_train['mths_since_issue_d:76-86'] = np.where((X_train['mths_since_issue_d'].isin(range(76,87))), 1, 0)
X_train['mths_since_issue_d:87-125'] = np.where((X_train['mths_since_issue_d'].isin(range(87,126))), 1, 0)
X_train['mths_since_issue_d:>126'] = np.where((X_train['mths_since_issue_d'].isin(range(126,
                                                        int(X_train['mths_since_issue_d'].max())))), 1, 0)

#### interest rate

In [94]:
# df_prep_temp = X_train
# df_prep_temp['int_rate_factor'] = pd.cut(df_prep_temp['int_rate'], 50)

In [95]:
# df_temp = woe_ordered_continuous(df_prep_temp, 'int_rate_factor', y_train)
# df_temp

In [96]:
# plot_by_woe(df_temp, 90)

In [97]:
X_train['int_rate:<7.878'] = np.where((X_train['int_rate'] <= 7.878), 1, 0)
X_train['int_rate:7.878-10.96'] = np.where(((X_train['int_rate'] > 7.878) & (X_train['int_rate'] <= 10.96)), 1, 0)
X_train['int_rate:10.96-13.014'] = np.where(((X_train['int_rate'] > 10.96) & (X_train['int_rate'] <= 13.014)), 1, 0)
X_train['int_rate:13.014-15.068'] = np.where(((X_train['int_rate'] > 13.014) & (X_train['int_rate'] <= 15.068)), 1, 0)
X_train['int_rate:15.068-20.718'] = np.where(((X_train['int_rate'] > 15.068) & (X_train['int_rate'] <= 20.718)), 1, 0)
X_train['int_rate:>20.718'] = np.where((X_train['int_rate'] > 20.718), 1, 0)

#### funded amount

In [98]:
# df_prep_temp = X_train
# df_prep_temp['funded_amnt_factor'] = pd.cut(df_prep_temp['funded_amnt'], 50)

In [99]:
# df_temp = woe_ordered_continuous(df_prep_temp, 'funded_amnt_factor', y_train)
# df_temp

In [100]:
# plot_by_woe(df_temp, 90)

too much variation. drop this feature so no need to create dummies

#### annual income

In [101]:
# X_train['annual_inc_factor'] = pd.cut(X_train['annual_inc'], 100)

In [102]:
# df_temp = woe_ordered_continuous(X_train, 'annual_inc_factor', y_train)
# df_temp

In [103]:
# df_prep_temp = X_train.loc[X_train['annual_inc'] <= 252573.72,:]

In [104]:
# df_prep_temp['annual_inc_factor'] = pd.cut(df_prep_temp['annual_inc'], 50)
# df_temp = woe_ordered_continuous(df_prep_temp, 'annual_inc_factor', y_train[df_prep_temp.index])
# df_temp

In [105]:
# plot_by_woe(df_temp, 90)

In [106]:
X_train['annual_inc:<10k'] = np.where((X_train['annual_inc'] <= 10100), 1, 0)
X_train['annual_inc:10k-45k'] = np.where(((X_train['annual_inc'] > 10100) & (X_train['annual_inc'] <= 45450.0)), 1, 0)
X_train['annual_inc:45k-55k'] = np.where(((X_train['annual_inc'] > 45450) & (X_train['annual_inc'] <= 55550)), 1, 0)
X_train['annual_inc:55k-60k'] = np.where(((X_train['annual_inc'] > 55550) & (X_train['annual_inc'] <= 60600)), 1, 0)
X_train['annual_inc:60k-70k'] = np.where(((X_train['annual_inc'] > 60600) & (X_train['annual_inc'] <= 70700)), 1, 0)
X_train['annual_inc:70k-85k'] = np.where(((X_train['annual_inc'] > 70700) & (X_train['annual_inc'] <= 85850)), 1, 0)
X_train['annual_inc:85k-101k'] = np.where(((X_train['annual_inc'] > 85850) & (X_train['annual_inc'] <= 101000)), 1, 0)
X_train['annual_inc:101k-111k'] = np.where(((X_train['annual_inc'] > 101000) & (X_train['annual_inc'] <= 111100)), 1, 0)
X_train['annual_inc:111k-131k'] = np.where(((X_train['annual_inc'] > 111100) & (X_train['annual_inc'] <= 131300)), 1, 0)
X_train['annual_inc:131k-161k'] = np.where(((X_train['annual_inc'] > 131300) & (X_train['annual_inc'] <= 161600)), 1, 0)
X_train['annual_inc:>161k'] = np.where((X_train['annual_inc'] > 161600), 1, 0)

#### months since last delinquency

In [107]:
# df_prep_temp = X_train[pd.notnull(X_train['mths_since_last_delinq'])]

In [108]:
# df_prep_temp['mths_since_last_delinq_factor'] = pd.cut(df_prep_temp['mths_since_last_delinq'],50)

In [109]:
# df_temp = woe_ordered_continuous(df_prep_temp, 'mths_since_last_delinq_factor', y_train[df_prep_temp.index])
# df_temp

In [110]:
# plot_by_woe(df_temp.iloc[:-24,:], 90)

In [111]:
#plot_by_woe(df_temp, 90)

In [112]:
X_train['mths_since_last_delinq:Missing'] = np.where((X_train['mths_since_last_delinq'].isnull()), 1, 0)
X_train['mths_since_last_delinq:0-3'] = np.where(((X_train['mths_since_last_delinq'] >= 0) & (X_train['mths_since_last_delinq'] < 4)), 1, 0)
X_train['mths_since_last_delinq:4-15'] = np.where(((X_train['mths_since_last_delinq'] >= 4) & (X_train['mths_since_last_delinq'] < 16)), 1, 0)
X_train['mths_since_last_delinq:16-31'] = np.where(((X_train['mths_since_last_delinq'] >= 16) & (X_train['mths_since_last_delinq'] < 32)), 1, 0)
X_train['mths_since_last_delinq:32-81'] = np.where(((X_train['mths_since_last_delinq'] >= 32) & (X_train['mths_since_last_delinq'] < 82)), 1, 0)
X_train['mths_since_last_delinq:>82'] = np.where((X_train['mths_since_last_delinq'] >= 82), 1, 0)

## Preprocessing Test Dataset

#### output the train set

In [113]:
loan_data_inputs_2015 = X_train
loan_data_targets_2015 = y_train

In [114]:
category_cols = [col for col in loan_data_inputs_2015.columns if loan_data_inputs_2015[col].dtypes == 'category']
loan_data_inputs_2015 = loan_data_inputs_2015.drop(category_cols, axis=1)

In [115]:
loan_data_inputs_2015.shape

(466285, 263)

In [116]:
loan_data_targets_2015.shape

(466285,)

#### save the output data

In [117]:
loan_data_inputs_2015.to_csv(data_path + "/" + "loan_data_inputs_2015.csv")
loan_data_targets_2015.to_csv(data_path + "/" + "loan_data_targets_2015.csv")

#### load the inputs train data

In [118]:
inputs_train_with_ref_cat = pd.read_csv(data_path + "/" + "inputs_train_with_ref_cat.csv",
                                         index_col = 0)

In [119]:
list_w_ref_cat = [
    'term:36',
    'term:60',
    'emp_length:0',
    'emp_length:1',
    'emp_length:2-6',
    'emp_length:7-9',
    'emp_length:10',
    'mths_since_issue_d:<4',
    'mths_since_issue_d:4-6',
    'mths_since_issue_d:7-9',
    'mths_since_issue_d:10-12',
    'mths_since_issue_d:13-14',
    'mths_since_issue_d:15-17',
    'mths_since_issue_d:18-23',
    'mths_since_issue_d:24-28',
    'mths_since_issue_d:29-58',
    'mths_since_issue_d:59-75',
    'mths_since_issue_d:76-86',
    'mths_since_issue_d:87-125',
    'mths_since_issue_d:>126',
    'int_rate:<7.878',
    'int_rate:7.878-10.96',
    'int_rate:10.96-13.014',
    'int_rate:13.014-15.068',
    'int_rate:15.068-20.718',
    'int_rate:>20.718',
    'annual_inc:<10k',
    'annual_inc:10k-45k',
    'annual_inc:45k-55k',
    'annual_inc:55k-60k',
    'annual_inc:60k-70k',
    'annual_inc:70k-85k',
    'annual_inc:85k-101k',
    'annual_inc:101k-111k',
    'annual_inc:111k-131k',
    'annual_inc:131k-161k',
    'annual_inc:>161k',
    'mths_since_last_delinq:Missing',
    'mths_since_last_delinq:0-3',
    'mths_since_last_delinq:4-15',
    'mths_since_last_delinq:16-31',
    'mths_since_last_delinq:32-81',
    'mths_since_last_delinq:>82',
    'grade:A',
    'grade:B',
    'grade:C',
    'grade:D',
    'grade:E',
    'grade:F',
    'grade:G',
    'home_ownership:RENT_OTHER_NONE_ANY',
    'home_ownership:OWN',
    'home_ownership:MORTGAGE',
    'addr_state:NV_NM_HI_SD',
    'addr_state:MO_MD_NC',
    'addr_state:PA_KY_TN',
    'addr_state:NJ_MI_IN',
    'addr_state:VA_MN_AK',
    'addr_state:AZ_NE_OH',
    'addr_state:DE_MA_UT_WI',
    'addr_state:GA_RI_WY_IL_MT_KS'
]

In [120]:
inputs_2015_with_ref_cat = loan_data_inputs_2015.loc[:, list_w_ref_cat]

In [121]:
inputs_train_with_ref_cat.shape

(1808534, 61)

In [122]:
inputs_2015_with_ref_cat.shape

(466285, 61)

#### calculating scores for train and 2015 data

In [123]:
df_scorecard = pd.read_csv(data_path + "/" + "df_scorecard.csv", index_col = 0)

In [124]:
df_scorecard

,index,Feature name,Coefficients,p_values,Original feature name,Score-Calculation,Score-Preliminary,check,Score-Final
0,0,Intercept,-0.520721,NaN,Intercept,304.830808,305.0,0.169192,305.0
1,1,term:36,0.129431,4.186576e-123,term,8.907534,9.0,0.092466,9.0
2,2,emp_length:1,0.134810,1.092466e-35,emp_length,9.277678,9.0,-0.277678,9.0
3,3,emp_length:2-6,0.168710,2.582206e-120,emp_length,11.610743,12.0,0.389257,12.0
4,4,emp_length:7-9,0.143064,1.235521e-56,emp_length,9.845738,10.0,0.154262,10.0
5,5,emp_length:10,0.197722,2.440321e-154,emp_length,13.607324,14.0,0.392676,14.0
6,6,mths_since_issue_d:<4,4.040152,0.000000e+00,mths_since_issue_d,278.045448,278.0,-0.045448,278.0
7,7,mths_since_issue_d:4-6,2.711620,0.000000e+00,mths_since_issue_d,186.615171,187.0,0.384829,187.0
8,8,mths_since_issue_d:7-9,1.780522,0.000000e+00,mths_since_issue_d,122.536516,123.0,0.463484,123.0
9,9,mths_since_issue_d:10-12,1.387675,0.000000e+00,mths_since_issue_d,95.500518,96.0,0.499482,96.0


1. add intercept

2. order the columns in the same way as the feature names from the scorecard data frame

In [125]:
inputs_train_with_ref_cat_w_intercept = inputs_train_with_ref_cat
inputs_train_with_ref_cat_w_intercept.insert(0,'Intercept',1)
inputs_train_with_ref_cat_w_intercept = inputs_train_with_ref_cat_w_intercept[df_scorecard['Feature name'].values]
inputs_train_with_ref_cat_w_intercept.head()

,Intercept,term:36,emp_length:1,emp_length:2-6,emp_length:7-9,emp_length:10,mths_since_issue_d:<4,mths_since_issue_d:4-6,mths_since_issue_d:7-9,mths_since_issue_d:10-12,mths_since_issue_d:13-14,mths_since_issue_d:15-17,mths_since_issue_d:18-23,mths_since_issue_d:24-28,mths_since_issue_d:29-58,mths_since_issue_d:59-75,mths_since_issue_d:76-86,mths_since_issue_d:87-125,int_rate:<7.878,int_rate:7.878-10.96,int_rate:10.96-13.014,int_rate:13.014-15.068,int_rate:15.068-20.718,annual_inc:<10k,annual_inc:45k-55k,annual_inc:55k-60k,annual_inc:60k-70k,annual_inc:70k-85k,annual_inc:85k-101k,annual_inc:101k-111k,annual_inc:111k-131k,annual_inc:131k-161k,annual_inc:>161k,mths_since_last_delinq:Missing,mths_since_last_delinq:4-15,mths_since_last_delinq:16-31,mths_since_last_delinq:32-81,mths_since_last_delinq:>82,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,home_ownership:OWN,home_ownership:MORTGAGE,addr_state:MO_MD_NC,addr_state:PA_KY_TN,addr_state:NJ_MI_IN,addr_state:VA_MN_AK,addr_state:AZ_NE_OH,addr_state:DE_MA_UT_WI,addr_state:GA_RI_WY_IL_MT_KS,term:60,emp_length:0,mths_since_issue_d:>126,int_rate:>20.718,annual_inc:10k-45k,mths_since_last_delinq:0-3,grade:G,home_ownership:RENT_OTHER_NONE_ANY,addr_state:NV_NM_HI_SD
305053,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0
50247,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0
1630001,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
927626,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
758488,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [126]:
inputs_2015_with_ref_cat_w_intercept = inputs_2015_with_ref_cat
inputs_2015_with_ref_cat_w_intercept.insert(0,'Intercept',1)
inputs_2015_with_ref_cat_w_intercept = inputs_2015_with_ref_cat_w_intercept[df_scorecard['Feature name'].values]
inputs_2015_with_ref_cat_w_intercept.head()

,Intercept,term:36,emp_length:1,emp_length:2-6,emp_length:7-9,emp_length:10,mths_since_issue_d:<4,mths_since_issue_d:4-6,mths_since_issue_d:7-9,mths_since_issue_d:10-12,mths_since_issue_d:13-14,mths_since_issue_d:15-17,mths_since_issue_d:18-23,mths_since_issue_d:24-28,mths_since_issue_d:29-58,mths_since_issue_d:59-75,mths_since_issue_d:76-86,mths_since_issue_d:87-125,int_rate:<7.878,int_rate:7.878-10.96,int_rate:10.96-13.014,int_rate:13.014-15.068,int_rate:15.068-20.718,annual_inc:<10k,annual_inc:45k-55k,annual_inc:55k-60k,annual_inc:60k-70k,annual_inc:70k-85k,annual_inc:85k-101k,annual_inc:101k-111k,annual_inc:111k-131k,annual_inc:131k-161k,annual_inc:>161k,mths_since_last_delinq:Missing,mths_since_last_delinq:4-15,mths_since_last_delinq:16-31,mths_since_last_delinq:32-81,mths_since_last_delinq:>82,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,home_ownership:OWN,home_ownership:MORTGAGE,addr_state:MO_MD_NC,addr_state:PA_KY_TN,addr_state:NJ_MI_IN,addr_state:VA_MN_AK,addr_state:AZ_NE_OH,addr_state:DE_MA_UT_WI,addr_state:GA_RI_WY_IL_MT_KS,term:60,emp_length:0,mths_since_issue_d:>126,int_rate:>20.718,annual_inc:10k-45k,mths_since_last_delinq:0-3,grade:G,home_ownership:RENT_OTHER_NONE_ANY,addr_state:NV_NM_HI_SD
0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,1,0
2,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0
3,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0


3. reshape scores

In [127]:
scorecard_scores = df_scorecard['Score-Final']
scorecard_scores = scorecard_scores.values.reshape(62,1)

4. get dot product of scores and inputs

In [128]:
y_scores_train = inputs_train_with_ref_cat_w_intercept.dot(scorecard_scores)
y_scores_train.head()

,0
305053,509.0
50247,680.0
1630001,506.0
927626,462.0
758488,469.0


In [129]:
y_scores_2015 = inputs_2015_with_ref_cat_w_intercept.dot(scorecard_scores)
y_scores_2015.head()

,0
0,483.0
1,412.0
2,435.0
3,437.0
4,457.0


5. concatenate to their respective main data frame

In [130]:
inputs_train_with_ref_cat_w_intercept = pd.concat([inputs_train_with_ref_cat_w_intercept,y_scores_train],
                                                 axis = 1)
inputs_2015_with_ref_cat_w_intercept = pd.concat([inputs_2015_with_ref_cat_w_intercept,y_scores_2015],
                                                 axis = 1)

In [131]:
# rename the column name
inputs_train_with_ref_cat_w_intercept.columns.values[inputs_train_with_ref_cat_w_intercept.shape[1] - 1] = 'Score'
inputs_2015_with_ref_cat_w_intercept.columns.values[inputs_2015_with_ref_cat_w_intercept.shape[1] - 1] = 'Score'

In [132]:
inputs_2015_with_ref_cat_w_intercept.head()

,Intercept,term:36,emp_length:1,emp_length:2-6,emp_length:7-9,emp_length:10,mths_since_issue_d:<4,mths_since_issue_d:4-6,mths_since_issue_d:7-9,mths_since_issue_d:10-12,mths_since_issue_d:13-14,mths_since_issue_d:15-17,mths_since_issue_d:18-23,mths_since_issue_d:24-28,mths_since_issue_d:29-58,mths_since_issue_d:59-75,mths_since_issue_d:76-86,mths_since_issue_d:87-125,int_rate:<7.878,int_rate:7.878-10.96,int_rate:10.96-13.014,int_rate:13.014-15.068,int_rate:15.068-20.718,annual_inc:<10k,annual_inc:45k-55k,annual_inc:55k-60k,annual_inc:60k-70k,annual_inc:70k-85k,annual_inc:85k-101k,annual_inc:101k-111k,annual_inc:111k-131k,annual_inc:131k-161k,annual_inc:>161k,mths_since_last_delinq:Missing,mths_since_last_delinq:4-15,mths_since_last_delinq:16-31,mths_since_last_delinq:32-81,mths_since_last_delinq:>82,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,home_ownership:OWN,home_ownership:MORTGAGE,addr_state:MO_MD_NC,addr_state:PA_KY_TN,addr_state:NJ_MI_IN,addr_state:VA_MN_AK,addr_state:AZ_NE_OH,addr_state:DE_MA_UT_WI,addr_state:GA_RI_WY_IL_MT_KS,term:60,emp_length:0,mths_since_issue_d:>126,int_rate:>20.718,annual_inc:10k-45k,mths_since_last_delinq:0-3,grade:G,home_ownership:RENT_OTHER_NONE_ANY,addr_state:NV_NM_HI_SD,Score
0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,483.0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,1,0,412.0
2,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,435.0
3,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,437.0
4,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,457.0


6. create categorical dummy variables for scores

In [133]:
inputs_train_with_ref_cat_w_intercept['Score:300-350'] = np.where((inputs_train_with_ref_cat_w_intercept['Score']>=300) & (inputs_train_with_ref_cat_w_intercept['Score']<350),1,0)
inputs_train_with_ref_cat_w_intercept['Score:350-400'] = np.where((inputs_train_with_ref_cat_w_intercept['Score']>=350) & (inputs_train_with_ref_cat_w_intercept['Score']<400),1,0)
inputs_train_with_ref_cat_w_intercept['Score:400-450'] = np.where((inputs_train_with_ref_cat_w_intercept['Score']>=400) & (inputs_train_with_ref_cat_w_intercept['Score']<450),1,0)
inputs_train_with_ref_cat_w_intercept['Score:450-500'] = np.where((inputs_train_with_ref_cat_w_intercept['Score']>=450) & (inputs_train_with_ref_cat_w_intercept['Score']<500),1,0)
inputs_train_with_ref_cat_w_intercept['Score:500-550'] = np.where((inputs_train_with_ref_cat_w_intercept['Score']>=500) & (inputs_train_with_ref_cat_w_intercept['Score']<550),1,0)
inputs_train_with_ref_cat_w_intercept['Score:550-600'] = np.where((inputs_train_with_ref_cat_w_intercept['Score']>=550) & (inputs_train_with_ref_cat_w_intercept['Score']<600),1,0)
inputs_train_with_ref_cat_w_intercept['Score:600-650'] = np.where((inputs_train_with_ref_cat_w_intercept['Score']>=600) & (inputs_train_with_ref_cat_w_intercept['Score']<650),1,0)
inputs_train_with_ref_cat_w_intercept['Score:650-700'] = np.where((inputs_train_with_ref_cat_w_intercept['Score']>=650) & (inputs_train_with_ref_cat_w_intercept['Score']<700),1,0)
inputs_train_with_ref_cat_w_intercept['Score:700-750'] = np.where((inputs_train_with_ref_cat_w_intercept['Score']>=700) & (inputs_train_with_ref_cat_w_intercept['Score']<750),1,0)
inputs_train_with_ref_cat_w_intercept['Score:750-800'] = np.where((inputs_train_with_ref_cat_w_intercept['Score']>=750) & (inputs_train_with_ref_cat_w_intercept['Score']<800),1,0)
inputs_train_with_ref_cat_w_intercept['Score:800-850'] = np.where((inputs_train_with_ref_cat_w_intercept['Score']>=800) & (inputs_train_with_ref_cat_w_intercept['Score']<=850),1,0)

In [134]:
inputs_2015_with_ref_cat_w_intercept['Score:300-350'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score']>=300) & (inputs_2015_with_ref_cat_w_intercept['Score']<350),1,0)
inputs_2015_with_ref_cat_w_intercept['Score:350-400'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score']>=350) & (inputs_2015_with_ref_cat_w_intercept['Score']<400),1,0)
inputs_2015_with_ref_cat_w_intercept['Score:400-450'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score']>=400) & (inputs_2015_with_ref_cat_w_intercept['Score']<450),1,0)
inputs_2015_with_ref_cat_w_intercept['Score:450-500'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score']>=450) & (inputs_2015_with_ref_cat_w_intercept['Score']<500),1,0)
inputs_2015_with_ref_cat_w_intercept['Score:500-550'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score']>=500) & (inputs_2015_with_ref_cat_w_intercept['Score']<550),1,0)
inputs_2015_with_ref_cat_w_intercept['Score:550-600'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score']>=550) & (inputs_2015_with_ref_cat_w_intercept['Score']<600),1,0)
inputs_2015_with_ref_cat_w_intercept['Score:600-650'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score']>=600) & (inputs_2015_with_ref_cat_w_intercept['Score']<650),1,0)
inputs_2015_with_ref_cat_w_intercept['Score:650-700'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score']>=650) & (inputs_2015_with_ref_cat_w_intercept['Score']<700),1,0)
inputs_2015_with_ref_cat_w_intercept['Score:700-750'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score']>=700) & (inputs_2015_with_ref_cat_w_intercept['Score']<750),1,0)
inputs_2015_with_ref_cat_w_intercept['Score:750-800'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score']>=750) & (inputs_2015_with_ref_cat_w_intercept['Score']<800),1,0)
inputs_2015_with_ref_cat_w_intercept['Score:800-850'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score']>=800) & (inputs_2015_with_ref_cat_w_intercept['Score']<=850),1,0)

# Population Stability Index: Calculation and Interpretation

In [135]:
PSI_calc_train = inputs_train_with_ref_cat_w_intercept.sum() / inputs_train_with_ref_cat_w_intercept.shape[0]

In [136]:
PSI_calc_2015 = inputs_2015_with_ref_cat_w_intercept.sum() / inputs_2015_with_ref_cat_w_intercept.shape[0]

In [137]:
PSI_calc = pd.concat([PSI_calc_train,PSI_calc_2015], axis=1)

In [138]:
PSI_calc = PSI_calc.reset_index()
PSI_calc['Original feature name'] = PSI_calc['index'].str.split(':').str[0]
PSI_calc.columns = ['index','Proportions_Train','Proportions_New','Original feature name']

In [139]:
# re-arrange columns order
PSI_calc = PSI_calc[np.array(['index','Original feature name','Proportions_Train','Proportions_New'])]

In [140]:
PSI_calc

,index,Original feature name,Proportions_Train,Proportions_New
0,Intercept,Intercept,1.000000,1.000000
1,term:36,term,0.712223,0.724778
2,emp_length:1,emp_length,0.065605,0.063528
3,emp_length:2-6,emp_length,0.337681,0.349310
4,emp_length:7-9,emp_length,0.116689,0.142537
5,emp_length:10,emp_length,0.330893,0.321797
6,mths_since_issue_d:<4,mths_since_issue_d,0.056888,0.000000
7,mths_since_issue_d:4-6,mths_since_issue_d,0.056734,0.000000
8,mths_since_issue_d:7-9,mths_since_issue_d,0.057816,0.000000
9,mths_since_issue_d:10-12,mths_since_issue_d,0.047756,0.000000


In [141]:
# exclude the intercept and score column
PSI_calc = PSI_calc[(PSI_calc['index'] != 'Intercept') & (PSI_calc['index'] != 'Score')]

In [142]:
PSI_calc

,index,Original feature name,Proportions_Train,Proportions_New
1,term:36,term,0.712223,0.724778
2,emp_length:1,emp_length,0.065605,0.063528
3,emp_length:2-6,emp_length,0.337681,0.349310
4,emp_length:7-9,emp_length,0.116689,0.142537
5,emp_length:10,emp_length,0.330893,0.321797
6,mths_since_issue_d:<4,mths_since_issue_d,0.056888,0.000000
7,mths_since_issue_d:4-6,mths_since_issue_d,0.056734,0.000000
8,mths_since_issue_d:7-9,mths_since_issue_d,0.057816,0.000000
9,mths_since_issue_d:10-12,mths_since_issue_d,0.047756,0.000000
10,mths_since_issue_d:13-14,mths_since_issue_d,0.035567,0.000000


## Calculate PSI

1. If %Actual or %Expected is 0, set it to 0

2. Apply formula

In [143]:
# deal with cases if %Actual or %Expected is 0
PSI_calc['Contribution'] = np.where((PSI_calc['Proportions_Train'] == 0) | (PSI_calc['Proportions_New'] == 0), 0,
                        (PSI_calc['Proportions_New'] - PSI_calc['Proportions_Train']) * np.log(PSI_calc['Proportions_New'] / PSI_calc['Proportions_Train']))

In [144]:
PSI_calc

,index,Original feature name,Proportions_Train,Proportions_New,Contribution
1,term:36,term,0.712223,0.724778,2.193920e-04
2,emp_length:1,emp_length,0.065605,0.063528,6.681003e-05
3,emp_length:2-6,emp_length,0.337681,0.349310,3.937556e-04
4,emp_length:7-9,emp_length,0.116689,0.142537,5.172025e-03
5,emp_length:10,emp_length,0.330893,0.321797,2.535496e-04
6,mths_since_issue_d:<4,mths_since_issue_d,0.056888,0.000000,0.000000e+00
7,mths_since_issue_d:4-6,mths_since_issue_d,0.056734,0.000000,0.000000e+00
8,mths_since_issue_d:7-9,mths_since_issue_d,0.057816,0.000000,0.000000e+00
9,mths_since_issue_d:10-12,mths_since_issue_d,0.047756,0.000000,0.000000e+00
10,mths_since_issue_d:13-14,mths_since_issue_d,0.035567,0.000000,0.000000e+00


3. Sum contributions by original features

In [145]:
PSI_calc.groupby('Original feature name')['Contribution'].sum()

Original feature name
Score                     0.500202
addr_state                0.000733
annual_inc                0.011124
emp_length                0.010991
grade                     0.018547
home_ownership            0.005655
int_rate                  0.067754
mths_since_issue_d        0.704746
mths_since_last_delinq    0.002575
term                      0.000779
Name: Contribution, dtype: float64

## PSI Interpretation

PSI = 0 No difference

PSI < 0.1 Little to no difference

0.1 < PSI < 0.25 Little difference (No action is taken)

PSI > 0.25 Big difference (Action is taken)

PSI = 1 Absolute difference

scores are significantly different. serious concern.